# Shut the Box

## Step 0: Utils for Jupyter Notebooks

### Add button to Show/Hide code (to focus on outputs)

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show Code"></form>''')

### Add magic command to store python variables in csv file

In [2]:
# credit to sirex, https://github.com/ipython/ipython/issues/6701
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Step 1: Generate models

In [3]:
# Edit this cell to define parameters for your model...

boards = 6 # Number of boards to use
sides = 6 # Number of sides per die
die = 1 # Number of die to roll each round

max_score = boards * (boards+1) // 2

In [4]:
%%writetemplate ShutTheBox/params/stb_params_high.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 1

In [5]:
%%writetemplate ShutTheBox/params/stb_params_low.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 2

In [6]:
%%writetemplate ShutTheBox/params/stb_nondet_params.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 0

In [7]:
!mkdir test

A subdirectory or file test already exists.


In [8]:
model_spec = f"stb{boards}_{die}d{sides}"

!mkdir "ShutTheBox/models/$model_spec"
output_model_high = f"ShutTheBox/models/{model_spec}/{model_spec}_high.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_high ShutTheBox/params/stb_params_high.csv

Model generated successfully at ShutTheBox\models\stb6_1d6\stb6_1d6_high.prism


In [9]:
output_model_low = f"ShutTheBox/models/{model_spec}/{model_spec}_low.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_low ShutTheBox/params/stb_params_low.csv

Model generated successfully at ShutTheBox\models\stb6_1d6\stb6_1d6_low.prism


In [10]:
output_model_nondet = f"ShutTheBox/models/{model_spec}/{model_spec}_nondet.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_nondet ShutTheBox/params/stb_nondet_params.csv

Model generated successfully at ShutTheBox\models\stb6_1d6\stb6_1d6_nondet.prism


In [11]:
output_props = f"ShutTheBox/props/{model_spec}.props"

!python utils/pyprism.py ShutTheBox/stbprops.pyprism $output_props ShutTheBox/params/stb_params_high.csv

Model generated successfully at ShutTheBox\props\stb6_1d6.props


## Step 2: Check properties, run experiments, export data

### Does the game eventually always terminate?

We expect to see a value of approximately 1 here (it won't be exact because CSGs only support methods based on numerical convergence, so it may be slightly under or even over 1!

In [12]:
!prism $output_model_high $output_props -prop 1 | grep -E "(Model checking)|(Result)|(sec)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over ]
Reachable states exploration and model construction done in 0.093 secs.
Time for model construction: 0.111 seconds.
Precomputation took 0.035 seconds.
Time for model checking: 0.121 seconds.
Result: 0.9999999999999998 (value in the initial state)


### Probability of obtaining each score 

In [13]:
data_location = f"ShutTheBox/data/{model_spec}"

!mkdir "$data_location"

prob_score_high_file = f"{data_location}/{model_spec}_prob_score_high.csv"

!prism $output_model_high $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_high_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.018518518518518517 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.018518518518518517 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.037037037037037035 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.046296296296296294 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.041666666666666664 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Pr

In [14]:
prob_score_low_file = f"{data_location}/{model_spec}_prob_score_low.csv"

!prism $output_model_low $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_low_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(sec)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Reachable states exploration and model construction done in 0.069 secs.
Time for model construction: 0.082 seconds.
Precomputation took 0.017 seconds.
Time for model checking: 0.03 seconds.
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Precomputation took 0.008 seconds.
Time for model checking: 0.012 seconds.
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Precomputation took 0.008 seconds.
Time for model checking: 0.016 seconds.
Result: 0.07407407407407407 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Precomputation took 0.008 seconds.
Time for model checking: 0.013 seconds.
Result: 0.06481481481481481 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&scor

In [15]:
prob_score_nondet_min_file = f"{data_location}/{model_spec}_prob_score_nondet_min.csv"

!prism $output_model_nondet $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_nondet_min_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.018518518518518517 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.009259259259259259 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.018518518518518517 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.037037037037037035 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.020833333333333332 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score=k ]
Pr

In [16]:
prob_score_nondet_max_file = f"{data_location}/{model_spec}_prob_score_nondet_max.csv"

!prism $output_model_nondet $output_props -prop 3 -const k=1:{max_score} -exportresults {prob_score_nondet_max_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.07407407407407407 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.07407407407407407 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.09259259259259259 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.16666666666666669 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.09722222222222224 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score=k ]
Propert

### Expected number of die rolls

In [17]:
print("High board strategy:")
!prism $output_model_high $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"
print("Low board strategy:")
!prism $output_model_low $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"

High board strategy:
Model checking: <<p1>>R{"no_rolls"}max=? [ F game_over ]
Result: 4.313786008230452 (value in the initial state)
Low board strategy:
Model checking: <<p1>>R{"no_rolls"}max=? [ F game_over ]
Result: 3.214077503429355 (value in the initial state)


### Probability of obtaining a score given a particular board is covered

In [18]:
for covered_board in range(1, boards+1):
    !prism $output_model_high $output_props -prop {10+covered_board} -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_score_given_{covered_board}.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 1.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 1.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 1.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.5 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b1=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.8 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_ov

Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.5 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&b4=1&score=k ]/<<p1>>Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.2 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_ov

### Cumulative probability of a score

In [19]:
!prism $output_model_high $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_high.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.05555555555555555 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.07407407407407407 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.09259259259259259 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.12962962962962965 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.1759259259259259 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.21759259259259253 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
P

In [20]:
!prism $output_model_low $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_low.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.05555555555555555 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.12962962962962962 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.19444444444444442 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.2685185185185185 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.43055555555555547 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.4791666666666665 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Pr

In [21]:
!prism $output_model_nondet $output_props -prop 5 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_min.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.05555555555555555 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.07407407407407407 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.09259259259259259 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.12962962962962965 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.1759259259259259 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.2175925925925926 (value in the initial state)
Model checking: <<p1>>Pmin=? [ F game_over&score<=k ]
Pr

In [22]:
cum_prob_max_file = "ShutTheBox/data/cum_prob_max.csv"

!prism $output_model_nondet $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_max.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.05555555555555555 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.12962962962962962 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.19444444444444445 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.2685185185185185 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.4305555555555554 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.4791666666666666 (value in the initial state)
Model checking: <<p1>>Pmax=? [ F game_over&score<=k ]
Pro